# Container Apps - File share Deployment
This template sets up a containerized application environment with necessary Azure resources, including a file share linked to the container's volumn mount, Azure Container Registry access, container deployment, storage account, and specified role assignments. Please ensure an Container Environment is created before proceeding with the deployment, see [/scenario-container-app-environment.ipynb](../environment/scenario-container-app-environment.ipynb).

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- Container app environment ([See scenario-container-app-environment.ipynb](../environment/scenario-container-app-environment.ipynb))


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [ ]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=eastus

2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "container-apps-deployment"
template_file = "main.bicep"
parameters_file = "main.bicepparam"

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

4. Create an Azure Resource group:

In [ ]:
!az login
!az account set --subscription  {subscription_id}
!az group create --name {resource_group} --location {location}

## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [ ]:
%%writefile {parameters_file}

using 'main.bicep'

param containerAppEnvName = 'my-container-app-env'
param containerAppEnvRG = 'container-apps'

param acrName = 'lenndewoltentestacr'
param acrRG = 'container-apps'

param containerAppName = 'my-test-container-app'
param storageAccountName = 'lenntstchtrappfilestrg'

param ingress = {
  external: true
  targetPort: 80
}

param containers = [
  {
    image: '${acrName}.azurecr.io/containerapps-helloworld:latest'
    name: 'hello-world-container'
    resources: {
      cpu: '0.25'
      memory: '0.5Gi'
    }
    env: [
      {
        name: 'FILE_SHARE_PATH'
        value: 'myfileshare'
      }
    ]
    volumeMounts: [
      {
        mountPath: 'myfileshare'
        volumeName: 'myfileshare'
      }
    ]
  }
]
param scale = {
  minReplicas: 0
  maxReplicas: 5
  rules: [
    {
      name: 'http-requests'
      http: {
        metadata: {
          concurrentRequests: '10'
        }
      }
    }
  ]
}

2. Optional: Seed container registry with the docker image

In [ ]:
acrName = 'lenndewoltentestacr'
repository = 'containerapps-helloworld'
tag = 'latest'

!az acr import --name {acrName} --source mcr.microsoft.com/azuredocs/containerapps-helloworld:latest --image {repository}:{tag}

3. Deploy the template:

In [ ]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name}


4. Open the app!

In [ ]:
import os
import webbrowser

fqdn = os.popen(f"az deployment group show --resource-group {resource_group} --name {deployment_name} --query \"properties.outputs.fqdn.value\" --output tsv").read().strip()

print(f"FQDN: {fqdn}")

webbrowser.open_new(fqdn)